# Tox21 mitochondria dataset

## Data Download

The Tox21 mitochondria dataset was downloaded from [tripod NIH Tox21 - tox21-mitotox-p1](https://tripod.nih.gov//tox21/pubdata/). Specifically, the zip file was downloaded and files were extracted. We used `tox21-mitotox-p1.aggregrated.txt` file to make the dataset.

According to its original paper, [Profiling of the Tox21 Chemical Collection for Mitochondrial Function to Identify Compounds that Acutely Decrease Mitochondrial Membrane Potential](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4286281/), we will use `ASSAY_OUTCOME` column as target instead of `CHANNEL_OUTCOME` column in the file, as it is the final MMP activity outcome based on its multi-channel readout (ratio, rhodamine, FITC, and cell viability).

The MMP experiment was done in HepG2 cell line, which is human hepatoma (liver cells) that is commonly used in drug hepatotoxicity studies.

## Setup

In [ ]:
!pip install rdkit -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 30.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd

from rdkit import Chem
from rdkit.Chem import SaltRemover, MolStandardize

In [ ]:
def preprocess_smiles(smiles):
    try:
        # Convert to a molecule object
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            return None

        # Standardization, get largest fragment
        lfc = MolStandardize.fragment.LargestFragmentChooser()
        mol = lfc.choose(mol)

        # Normalize
        norm = MolStandardize.normalize.Normalizer()
        mol = norm.normalize(mol)

        # # Desalt, duplicates with largest fragment
        # remover = SaltRemover.SaltRemover()
        # mol = remover.StripMol(mol, dontRemoveEverything=True)

        # Neutralization
        uncharger = MolStandardize.charge.Uncharger()
        mol = uncharger.uncharge(mol)

        # Convert back to SMILES
        standardized_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
        return standardized_smiles
    except Exception as e:
        print(f"An error occurred for SMILES {smiles}: {e}")
        return None

## Data Process

In [ ]:
df = pd.read_csv('tox21-mitotox-p1_aggregrated.csv')

In [ ]:
# remove salt and keep the large fragment of the smiles
smi = pd.DataFrame(df.SMILES.drop_duplicates()) # as there are many duplicated smiles, it's better to remove them
smi['smiles'] = smi.SMILES.apply(preprocess_smiles)

df = df.merge(smi) # merge the converted smiles to the original dataframe
df = df.dropna(subset='smiles').reset_index(drop=True)

In [ ]:
def get_unique(x): return x.unique()

In [ ]:
# groupby smiles, as compared with SAMPLE_NAME, and TOX21_ID, it has less duplicates
dd = df.groupby('smiles').agg({'SAMPLE_NAME':get_unique,
                           'TOX21_ID':get_unique,
                           'ASSAY_OUTCOME': get_unique,
                           'CHANNEL_OUTCOME':get_unique })

In [ ]:
dd['TOX21_ID_len'] = dd.TOX21_ID.apply(len)
dd['ASSAY_OUTCOME_len'] = dd.ASSAY_OUTCOME.apply(len)
dd['CHANNEL_OUTCOME_len'] = dd.CHANNEL_OUTCOME.apply(len)
dd['SAMPLE_NAME_len'] = dd.SAMPLE_NAME.apply(len)

In [ ]:
dd.ASSAY_OUTCOME_len.value_counts()

1    6808
2     583
3      50
4       4
5       1
Name: ASSAY_OUTCOME_len, dtype: int64

In [ ]:
# remove compounds that has ASSAY_OUTCOME_len more than 1
d = dd.query('ASSAY_OUTCOME_len == 1').copy()

In [ ]:
d.ASSAY_OUTCOME.str[0].value_counts()

inactive                               4626
active antagonist                       750
inconclusive                            603
inconclusive agonist                    245
active agonist                          207
inconclusive antagonist (cytotoxic)     198
inconclusive antagonist                 174
inconclusive agonist (cytotoxic)          5
Name: ASSAY_OUTCOME, dtype: int64

In [ ]:
d['ASSAY_OUTCOME'] = d['ASSAY_OUTCOME'].str[0]

In [ ]:
d = d.rename(columns={'smiles':'SMILES'})

In [ ]:
ds = d.reset_index()[['SMILES','SAMPLE_NAME','TOX21_ID','ASSAY_OUTCOME','TOX21_ID_len','SAMPLE_NAME_len']]

In [ ]:
ds.sort_values('SAMPLE_NAME_len').tail(1).SAMPLE_NAME.values # these chemicals have same smiles (large fragment)

array([array(['1-Butylpyridinium hexafluorophosphate',
              '1-Butylpyridinium bromide', '1-Butylpyridinium iodide',
              '1-Butylpyridinium tetrafluoroborate',
              '1-Butylpyridinium trifluoromethanesulfonate',
              'N-Butylpyridinium chloride'], dtype=object)            ],
      dtype=object)

In [ ]:
ds.to_parquet('tox21_MMP.parquet')

## To use

In [ ]:
ds.ASSAY_OUTCOME.value_counts()

inactive                               4626
active antagonist                       750
inconclusive                            603
inconclusive agonist                    245
active agonist                          207
inconclusive antagonist (cytotoxic)     198
inconclusive antagonist                 174
inconclusive agonist (cytotoxic)          5
Name: ASSAY_OUTCOME, dtype: int64

There are different categories of ASSAY_OUTCOME. According to the original paper,
- inactive: compounds that do not have any effect on mitochondrial membrane potential (MMP)
- active antagonist: compounds that decrease MMP, with the highest reproducibility
- active agonist: compounds that increase MMP, with lower reproducibility due to the experiment design (so consider drop these)
- inclusive: compounds have inconsistent results across different channel readouts.

For inclucsive antagonist/agonist, the paper did not explicit them very clearly, but it is possible that these compounds were shown to decrease/increase MMP, but has reverse effect on cell viability. They also have relative lower reproducibility.

To get a clear mitochondrial toxicity dataset, one can only keep `inactive` and `active antagonist`, and optionally, consider about taking `inconcusive antagonist` and `inconclusive antagonist (cytotoxic)`

In [ ]:
# get all antagonist, regardless of inconclusive
anta_idx = ds.ASSAY_OUTCOME.str.contains('antagonist')
inact_idx = ds.ASSAY_OUTCOME.str.contains('inactive')
comb = anta_idx|inact_idx

In [ ]:
final = ds.loc[comb].copy().reset_index(drop=True)

In [ ]:
final.ASSAY_OUTCOME.value_counts()

inactive                               4626
active antagonist                       750
inconclusive antagonist (cytotoxic)     198
inconclusive antagonist                 174
Name: ASSAY_OUTCOME, dtype: int64

In [ ]:
final['target'] = final.ASSAY_OUTCOME.apply(lambda x: 0 if x=='inactive' else 1)

In [ ]:
final

,SMILES,SAMPLE_NAME,TOX21_ID,ASSAY_OUTCOME,TOX21_ID_len,SAMPLE_NAME_len,target
0,Br,[Sodium bromide],[Tox21_301343],inactive,1,1,0
1,BrC(Br)Br,[Bromoform],[Tox21_200189],inactive,1,1,0
2,BrC(Br)C(Br)(Br)Br,[Pentabromoethane],[Tox21_200571],inactive,1,1,0
3,BrC(Br)C(Br)Br,"[1,1,2,2-Tetrabromoethane]",[Tox21_200364],inactive,1,1,0
4,BrC/C=C/CBr,"[(E)-1,4-Dibromo-2-butene]",[Tox21_202728],inactive,1,1,0
...,...,...,...,...,...,...,...
5743,c1cncc([C@@H]2CCCCN2)c1,[L-3-(2'-Piperidyl)pyridine],[Tox21_301952],inactive,1,1,0
5744,c1coc(Cn2cccc2)c1,[1-Furfurylpyrrole],[Tox21_302604],inactive,1,1,0
5745,c1csc(C2(N3CCCCC3)CCCCC2)c1,[Tenocyclidine],[Tox21_111833],inactive,1,1,0
5746,c1csc(SSc2cccs2)c1,[2-Thienyl disulfide],[Tox21_302478],inactive,1,1,0


In [ ]:
final.to_parquet('tox21_MMP_inactive_and_antagonist.parquet')

In [ ]:
# f2 = pd.read_parquet('tox21_MMP_inactive_and_antagonist.parquet')

## Compare with ML/DL benchmark tox21

In [ ]:
!pip install PyTDC -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 38.7 MB/s eta 0:00:00


In [ ]:
from tdc.utils import retrieve_label_name_list
label_list = retrieve_label_name_list('Tox21')

In [ ]:
label_list

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

We will take SR-MMP, as it is the same one we used.

In [ ]:
from tdc.single_pred import Tox
data = Tox(name = 'Tox21', label_name = 'SR_MMP')
split = data.get_split()

Found local copy...
Loading...
Done!


In [ ]:
full = pd.concat([split['train'],split['valid'],split['test']]).reset_index(drop=True)

In [ ]:
full = full.rename(columns = {'Drug':'SMILES'})

In [ ]:
full['SMILES2'] = full.SMILES.apply(preprocess_smiles)

[04:48:40] WARNING: not removing hydrogen atom without neighbors
[04:48:44] Can't kekulize mol.  Unkekulized atoms: 3 10


It is very hard to know what they are (e.g., antagonist/agonist) in the benchmark dataset, as there are few information.

In [ ]:
full.shape

(5810, 4)

In [ ]:
# Let's check the difference of smiles after standardization
full.loc[full.duplicated('SMILES2',keep=False)].sort_values('SMILES2')

,Drug_ID,SMILES,Y,SMILES2
266,TOX7835,C/C=C/C=C/C(=O)[O-],0.0,C/C=C/C=C/C(=O)O
5145,TOX1277,C/C=C/C=C/C(=O)O,0.0,C/C=C/C=C/C(=O)O
2808,TOX24823,C=C(C)C(=O)[O-],0.0,C=C(C)C(=O)O
1625,TOX5542,C=C(C)C(=O)O,0.0,C=C(C)C(=O)O
4775,TOX3349,C=CCN1CC[C@]23c4c5ccc(O)c4O[C@H]2C(=O)CC[C@@]3...,0.0,C=CCN1CC[C@]23c4c5ccc(O)c4O[C@H]2C(=O)CC[C@@]3...
...,...,...,...,...
4463,TOX5709,[Ni+2].c1cc[cH-]c1.c1cc[cH-]c1,0.0,c1cccc1
504,TOX5326,[Fe+2].c1cc[cH-]c1.c1cc[cH-]c1,0.0,c1cccc1
3016,TOX4615,[Cr+2].c1cc[cH-]c1.c1cc[cH-]c1,0.0,c1cccc1
2786,TOX12722,c1nc[n-]n1,0.0,c1nc[nH]n1


In [ ]:
full['SMILES'] = full['SMILES2']

In [ ]:
overlap = full.merge(final) # overlap with our dataset

In [ ]:
overlap.shape # we have these many overlap with the benchmark dataset

(5020, 10)

In [ ]:
overlap.apply(lambda r: r.Y==r.target,axis=1).value_counts() # the overlap datasets agree with each other

True    5020
dtype: int64

In [ ]:
overlap.ASSAY_OUTCOME.value_counts()
# it is mainly inactive and active antagonist, the benchmark did not include those inclucsive antagonist

inactive             4321
active antagonist     699
Name: ASSAY_OUTCOME, dtype: int64

In [ ]:
# check those that do not overlap with the benchmark, seems to be inclucsive antagonist
final.loc[~final.SMILES.isin(overlap.SMILES)].ASSAY_OUTCOME.value_counts()

inactive                               380
inconclusive antagonist (cytotoxic)    198
inconclusive antagonist                174
active antagonist                       58
Name: ASSAY_OUTCOME, dtype: int64